
# 📈 Dự đoán giá đóng cửa Bitcoin vào ngày hôm sau

**Môn học: Machine Learning**  
**Sinh viên: [Điền tên bạn]**  

Trong notebook này, chúng ta sẽ sử dụng các kỹ thuật học máy để dự đoán giá đóng cửa (`close`) của Bitcoin vào ngày hôm sau dựa trên dữ liệu lịch sử và các yếu tố kinh tế như lãi suất FED, chỉ số USD, v.v.


In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
import seaborn as sns


## 1. Đọc và tiền xử lý dữ liệu

In [ ]:

# Đọc dữ liệu
df = pd.read_csv("your_dataset.csv")  # Thay bằng tên file bạn
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values('timestamp').reset_index(drop=True)

# Tạo nhãn là close ngày hôm sau
df['target_close'] = df['close'].shift(-1)
df = df.dropna()

# Trích xuất đặc trưng thời gian
df['day'] = df['timestamp'].dt.day
df['month'] = df['timestamp'].dt.month
df['weekday'] = df['timestamp'].dt.weekday

# Tạo thêm một số đặc trưng mới
df['range_high_low'] = df['high'] - df['low']
df['change_open_close'] = df['close'] - df['open']
df['close_pct_change'] = df['close'].pct_change().fillna(0)


## 2. Chọn đặc trưng và chuẩn hóa

In [ ]:

# Lưu ý: 'year', 'month', 'weekday' là dữ liệu dạng thứ tự (ordinal), thường được chuẩn hóa nếu không dùng tree-based models.
features = [
    'open', 'high', 'low', 'close', 'volume', 'marketCap',
    'FEDFUNDS', 'US_Dollar_Price', 'day', 'month', 'weekday',
    'range_high_low', 'change_open_close', 'close_pct_change'
]

X = df[features]
y = df['target_close']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


## 3. Tách tập train/test

In [ ]:

# Không shuffle vì đây là dữ liệu thời gian
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, shuffle=False)


## 4. Huấn luyện và đánh giá các mô hình

In [ ]:

models = {
    "Gradient Boosting": GradientBoostingRegressor(n_estimators=100, random_state=42),
    "Random Forest": RandomForestRegressor(n_estimators=100, random_state=42),
    "Linear Regression": LinearRegression()
}

results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    results[name] = {"MSE": mse, "MAE": mae, "R2": r2}
    print(f"--- {name} ---")
    print(f"MSE: {mse:.2f} | MAE: {mae:.2f} | R2: {r2:.4f}\n")


## 5. So sánh kết quả dự đoán

In [ ]:

# Vẽ biểu đồ cho mô hình tốt nhất (Gradient Boosting)
best_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

plt.figure(figsize=(12, 5))
plt.plot(y_test.values, label='Giá thực tế')
plt.plot(y_pred, label='Giá dự đoán (GBR)')
plt.title('So sánh giá Close thực tế và dự đoán')
plt.xlabel("Ngày")
plt.ylabel("Giá Close")
plt.legend()
plt.show()



## ✅ Kết luận

- Dữ liệu được xử lý đầy đủ gồm chuẩn hóa, tạo đặc trưng động và thời gian.
- Gradient Boosting Regressor cho kết quả tốt nhất về sai số và độ chính xác.
- Có thể cải thiện thêm bằng cách:
  - Thêm đặc trưng chuỗi thời gian (`rolling mean`, `volatility`...)
  - Dùng mô hình học sâu như LSTM nếu dữ liệu dài hơn

**👉 Đây là một pipeline học máy đầy đủ, phù hợp để nộp cho bài tập môn học.**
